# Core 9 — Minimal Forecasting Models (core9_03)

본 노트북은 예측 성능(accuracy)을 최적화하지 않는다.

목적은 예측이 'fallback 실행'이 아니라
**fallback 예약(reservation)을 위한 운영 신호**로 유용한지를 검증하는 것이다.

평가 축:
- Lead time (사건 전에 얼마나 미리 경보가 떴는가)
- Alarm stability (경보 토글이 과도하지 않은가)
- Case separability (A_ALWAYS_ALLOW vs B_GOVERNED 분리)
- Governance cost (오탐/미탐 비용 기반)

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

FEATURES_PATH = Path("../artifact/core9/core9_02_features.csv")
TARGETS_PATH  = Path("../artifact/core9/core9_01_targets_preview.csv")

assert FEATURES_PATH.exists(), "core9_02_features.csv not found"
features = pd.read_csv(FEATURES_PATH)

if TARGETS_PATH.exists():
    targets = pd.read_csv(TARGETS_PATH)
    # 타깃이 features에 없으면 merge
    for c in ["y_soms_runaway","y_osc_nonrecover","y_conflict_selfamp"]:
        if c not in features.columns and c in targets.columns:
            features = features.merge(
                targets[["run_id","case_id","antibody_id","step",c]],
                on=["run_id","case_id","antibody_id","step"],
                how="left"
            )

# 기본 정렬
features = features.sort_values(["run_id","case_id","antibody_id","step"]).reset_index(drop=True)
features.head()

,run_id,case_id,antibody_id,step,blocked_rate_window_x,veto_streak_x,action_toggle_rate_x,SoMS_cumsum_window_x,recovery_time_est,pressure_index,...,toggle_over_th_streak_10,blocked_over_partialseal_streak_10,SoMS_cumsum_window_y,action_toggle_rate_y,blocked_rate_window_y,veto_streak_y,conflict_density_proxy,y_soms_runaway,y_osc_nonrecover,y_conflict_selfamp
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0,0.0,0,0.0,0.0,NaN,0.000,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,1,0.0,0,0.0,0.0,NaN,0.000,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,2,0.0,0,0.0,0.0,NaN,0.000,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,3,0.0,0,0.0,0.0,NaN,0.000,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,4,0.0,0,0.0,0.6,NaN,0.006,...,0,0,0.6,0.0,0.0,0.0,0.0,0,0,0


In [2]:
FORECAST_RULE_ID = "core9_03_v1"

# Baseline risk components (모델 없이)
# - risk_soms: 누적 압력 (pressure_index)
# - risk_osc: 토글 지속 (toggle_over_th_streak_10 기반)
# - risk_conflict: blocked + toggle + veto proxy (features에 conflict proxy 없으면 pressure로 대체)

# 존재하는 컬럼 확인
must_have = ["pressure_index", "step", "run_id", "case_id"]
missing = [c for c in must_have if c not in features.columns]
assert not missing, f"Missing required feature columns: {missing}"

# optional columns
has_toggle_streak = "toggle_over_th_streak_10" in features.columns
has_block_ma = "blocked_rate_ma_10" in features.columns # 모델 버전/스코어 정의 (Code)

In [3]:
df = features.copy()

# component 1: SoMS pressure
df["risk_score_soms"] = df["pressure_index"].clip(0, 1)

# component 2: oscillation (streak 기반, 없으면 toggle_rate_ma_10로 근사)
if has_toggle_streak:
    # streak 값이 클수록 위험 (10 step 기준으로 0~1 스케일)
    df["risk_score_osc"] = (df["toggle_over_th_streak_10"] / 10.0).clip(0, 1)
elif "toggle_rate_ma_10" in df.columns:
    # moving average를 0~1로 단순 정규화
    df["risk_score_osc"] = (df["toggle_rate_ma_10"]).clip(0, 1)
else:
    df["risk_score_osc"] = 0.0

# component 3: conflict proxy (blocked_rate_ma_10가 있으면 사용, 없으면 0)
if has_block_ma:
    df["risk_score_conflict"] = (df["blocked_rate_ma_10"]).clip(0, 1)
else:
    df["risk_score_conflict"] = 0.0

# total (가중치: 운영 안정성 위해 soms 비중 높게)
w_soms, w_osc, w_conf = 0.5, 0.3, 0.2
df["risk_score_total"] = (
    w_soms*df["risk_score_soms"] +
    w_osc*df["risk_score_osc"] +
    w_conf*df["risk_score_conflict"]
).clip(0, 1)

df[["risk_score_soms","risk_score_osc","risk_score_conflict","risk_score_total"]].describe() # Baseline 1: 규칙 기반 risk_score 생성

,risk_score_soms,risk_score_osc,risk_score_conflict,risk_score_total
count,180.000000,180.000000,126.000000,126.000000
mean,0.139514,0.030556,0.028651,0.103807
std,0.140677,0.143421,0.045802,0.092490
min,0.000000,0.000000,0.000000,0.000000
25%,0.016500,0.000000,0.000000,0.039375
50%,0.093250,0.000000,0.000000,0.074375
75%,0.231368,0.000000,0.050833,0.152497
max,0.521271,1.000000,0.170000,0.413241


In [4]:
# 임계치는 '예약 후보' 관점으로 보수적으로 설정
LOW_TH  = 0.30
MID_TH  = 0.55
HIGH_TH = 0.75

def hazard_level(x):
    if x >= HIGH_TH:
        return "HIGH"
    if x >= MID_TH:
        return "MID"
    if x >= LOW_TH:
        return "LOW"
    return "NONE"

df["forecast_hazard_level"] = df["risk_score_total"].map(hazard_level)
df["forecast_rule_id"] = FORECAST_RULE_ID

df[["risk_score_total","forecast_hazard_level"]].head(20) # Hazard Level (LOW/MID/HIGH) 정의

,risk_score_total,forecast_hazard_level
0,NaN,NONE
1,NaN,NONE
2,NaN,NONE
3,NaN,NONE
4,NaN,NONE
5,NaN,NONE
6,NaN,NONE
7,NaN,NONE
8,NaN,NONE
9,0.02725,NONE


In [5]:
# 타깃 우선순위: 사건 정의는 y_* 중 하나라도 1이면 "사건 발생"
target_cols = ["y_soms_runaway","y_osc_nonrecover","y_conflict_selfamp"]
for c in target_cols:
    if c not in df.columns:
        df[c] = 0

df["y_event_any"] = (
    df[target_cols].fillna(0).astype(int).max(axis=1)
)

# 경보 정의: MID 이상을 "예약 경보"로 간주 (HIGH면 강한 경보)
ALARM_LEVELS = {"MID", "HIGH"}
df["alarm_on"] = df["forecast_hazard_level"].isin(ALARM_LEVELS)

def compute_lead_time_for_group(g: pd.DataFrame) -> pd.DataFrame:
    """
    lead_time:
      - event_any가 1이 되는 최초 step을 찾고,
      - 그 이전에 alarm_on이 처음 켜진 step과의 차이를 계산
    """
    g = g.sort_values("step").copy()

    event_steps = g.loc[g["y_event_any"] == 1, "step"].to_list()
    if not event_steps:
        g["lead_time"] = np.nan
        g["first_event_step"] = np.nan
        g["first_alarm_step"] = np.nan
        return g

    first_event = int(event_steps[0])

    alarm_steps = g.loc[(g["alarm_on"] == True) & (g["step"] <= first_event), "step"].to_list()
    first_alarm = int(alarm_steps[0]) if alarm_steps else np.nan

    g["first_event_step"] = first_event
    g["first_alarm_step"] = first_alarm
    g["lead_time"] = (first_event - first_alarm) if alarm_steps else np.nan
    return g

df = (
    df.groupby(["run_id","case_id","antibody_id"], group_keys=False)
      .apply(compute_lead_time_for_group)
) # 운영 지표 1: Lead Time 계산 함수

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_20978/666997118.py:42: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_lead_time_for_group)


In [6]:
def compute_alarm_toggle_rate(g: pd.DataFrame) -> pd.Series:
    g = g.sort_values("step")
    s = g["alarm_on"].astype(int).to_numpy()
    if len(s) <= 1:
        return pd.Series({"alarm_toggle_rate": 0.0})

    toggles = np.abs(np.diff(s)).sum()
    # step 수 대비 토글 비율
    rate = toggles / (len(s) - 1)
    return pd.Series({"alarm_toggle_rate": float(rate)})

toggle_df = (
    df.groupby(["run_id","case_id","antibody_id"])
      .apply(compute_alarm_toggle_rate)
      .reset_index()
)

toggle_df.head() # 운영 지표 2: Alarm Stability

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_20978/876163728.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_alarm_toggle_rate)


,run_id,case_id,antibody_id,alarm_toggle_rate
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0.0
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_B,0.0
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_C,0.0
3,core7_04_1767776352,B_GOVERNED,antibody_A,0.0
4,core7_04_1767776352,B_GOVERNED,antibody_B,0.0


In [7]:
def bucket_case(case_id: str) -> str:
    if str(case_id).startswith("A_"):
        return "A_ALWAYS_ALLOW"
    if str(case_id).startswith("B_"):
        return "B_GOVERNED"
    return "OTHER"

case_summary = (
    df.groupby(["run_id","case_id"])
      .agg(
          any_alarm=("alarm_on","max"),
          max_risk=("risk_score_total","max"),
          max_level=("forecast_hazard_level", lambda x: "HIGH" if (x=="HIGH").any() else ("MID" if (x=="MID").any() else ("LOW" if (x=="LOW").any() else "NONE"))),
          any_event=("y_event_any","max"),
          lead_time=("lead_time","max"),
      )
      .reset_index()
)

case_summary["bucket"] = case_summary["case_id"].map(bucket_case)
case_summary # 운영 지표 3: A vs B 케이스 분리

,run_id,case_id,any_alarm,max_risk,max_level,any_event,lead_time,bucket
0,core7_04_1767776352,A_ALWAYS_ALLOW,False,0.120750,NONE,0,NaN,A_ALWAYS_ALLOW
1,core7_04_1767776352,B_GOVERNED,False,0.413241,LOW,0,NaN,B_GOVERNED


In [8]:
# (대체) lambda 없이 max_level 만들기
tmp = df[["run_id","case_id","forecast_hazard_level"]].copy()
tmp["is_high"] = tmp["forecast_hazard_level"].eq("HIGH")
tmp["is_mid"]  = tmp["forecast_hazard_level"].eq("MID")
tmp["is_low"]  = tmp["forecast_hazard_level"].eq("LOW")

level_summary = (
    tmp.groupby(["run_id","case_id"], as_index=False)
       .agg(is_high=("is_high","max"), is_mid=("is_mid","max"), is_low=("is_low","max"))
)

def pick_level(r):
    if r["is_high"]: return "HIGH"
    if r["is_mid"]:  return "MID"
    if r["is_low"]:  return "LOW"
    return "NONE"

level_summary["max_level"] = level_summary.apply(pick_level, axis=1)
case_summary = case_summary.drop(columns=["max_level"]).merge(level_summary[["run_id","case_id","max_level"]], on=["run_id","case_id"], how="left")
case_summary

,run_id,case_id,any_alarm,max_risk,any_event,lead_time,bucket,max_level
0,core7_04_1767776352,A_ALWAYS_ALLOW,False,0.120750,0,NaN,A_ALWAYS_ALLOW,NONE
1,core7_04_1767776352,B_GOVERNED,False,0.413241,0,NaN,B_GOVERNED,LOW


In [9]:
# 비용 정의(예시):
# - 오탐(Alarm=1, Event=0): 조기 닫기 비용
# - 미탐(Alarm=0, Event=1): 폭주 방치 비용 (더 큼)
COST_FP = 1.0
COST_FN = 5.0

case_summary["false_positive"] = ((case_summary["any_alarm"] == 1) & (case_summary["any_event"] == 0)).astype(int)
case_summary["false_negative"] = ((case_summary["any_alarm"] == 0) & (case_summary["any_event"] == 1)).astype(int)

case_summary["governance_cost"] = (
    COST_FP * case_summary["false_positive"] +
    COST_FN * case_summary["false_negative"]
)

case_summary[["run_id","case_id","bucket","any_alarm","any_event","false_positive","false_negative","governance_cost"]] #운영 지표 4: Governance Cost (간단 비용 함수)

,run_id,case_id,bucket,any_alarm,any_event,false_positive,false_negative,governance_cost
0,core7_04_1767776352,A_ALWAYS_ALLOW,A_ALWAYS_ALLOW,False,0,0,0,0.0
1,core7_04_1767776352,B_GOVERNED,B_GOVERNED,False,0,0,0,0.0


In [10]:
EXPORT_DIR = Path("../artifact/core9")
EXPORT_DIR.mkdir(exist_ok=True)

export_cols = [
    "run_id","case_id","antibody_id","step",
    "risk_score_soms","risk_score_osc","risk_score_conflict","risk_score_total",
    "forecast_hazard_level","forecast_rule_id",
    "alarm_on","y_event_any","first_alarm_step","first_event_step","lead_time"
]

export_df = df[export_cols].copy()
export_path = EXPORT_DIR / "core9_03_forecast_scores.csv"
export_df.to_csv(export_path, index=False)

# 요약도 같이 저장 (심사 친화)
summary_path = EXPORT_DIR / "core9_03_case_summary.csv"
case_summary.to_csv(summary_path, index=False)

toggle_path = EXPORT_DIR / "core9_03_alarm_toggle_rate.csv"
toggle_df.to_csv(toggle_path, index=False)

print("Exported:")
print("-", export_path)
print("-", summary_path)
print("-", toggle_path)

Exported:
- ../artifact/core9/core9_03_forecast_scores.csv
- ../artifact/core9/core9_03_case_summary.csv
- ../artifact/core9/core9_03_alarm_toggle_rate.csv


## Core 9 (core9_03) — Operational Usefulness, Not Accuracy

- 예측은 개입 명령이 아니라 “예약 판단 신호”입니다.
- 성능 지표 대신 운영 지표로 검증합니다.
  - Lead time 분포
  - Alarm toggle rate (안정성)
  - A_ALWAYS_ALLOW vs B_GOVERNED 분리성
  - Governance cost (FP vs FN 비용 기반)